Running a local nominatim server

docker run -it -v /var/tmp/ldh/nominatim:/data  -e PBF_PATH=/data/texas-latest.osm.pbf  -p 8080:8080   --name nominatimjyp1 mediagis/nominatim:5.1 

In [ ]:
!pip3 install geopy

In [4]:
import pickle
import pandas as pd

In [14]:
!date

Wed Jul  9 10:30:55 AM CDT 2025


In [15]:
headers = [
    "Taxpayer_Number", "Taxpayer_Name", "Taxpayer_Address", "Taxpayer_City", "Taxpayer_State",
    "Taxpayer_Zip", "Taxpayer_County", "Taxpayer_Phone", "Location_Number", "Location_Name",
    "Location_Address", "Location_City", "Location_State", "Location_Zip", "Location_County",
    "Location_Phone", "Unit_Capacity", "Responsibility_Begin_Date", "Responsibility_End_Date",
    "Obligation_End_Date", "Filer_Type", "Total_Room_Receipts", "Taxable_Receipts"
]

In [16]:
df = pd.read_csv(r'./Map_Chart_Datasets/HOT2505.CSV', low_memory=False, on_bad_lines='skip', encoding="latin-1", names=headers, header=0)

In [17]:
!date

Wed Jul  9 10:31:10 AM CDT 2025


In [18]:
from geopy.geocoders import Nominatim
from geopy.adapters import AioHTTPAdapter
import pandas as pd
import time


# Point to your local server
geolocator = Nominatim(
    user_agent="tx_hotels_map",
    domain="localhost:8080",
    scheme="http"
)

def get_coordinates(row):
    full_address = f"{row['Location_Address']}, {row['Location_City']}, TX {row['Location_Zip']}"
    try:
        location = geolocator.geocode(full_address, timeout=3)
#        time.sleep(1)  # Respect rate limits even locally
        if location:
            return pd.Series([location.latitude, location.longitude])
    except Exception as e:
        print(f"Error with address '{full_address}': {e}")
        return pd.Series([None, None])

df[["Latitude", "Longitude"]] = df.apply(get_coordinates, axis=1)


In [19]:
!date

Wed Jul  9 10:46:15 AM CDT 2025


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15837 entries, 0 to 15836
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Taxpayer_Number            15837 non-null  int64  
 1   Taxpayer_Name              15837 non-null  object 
 2   Taxpayer_Address           15837 non-null  object 
 3   Taxpayer_City              15837 non-null  object 
 4   Taxpayer_State             15837 non-null  object 
 5   Taxpayer_Zip               15837 non-null  object 
 6   Taxpayer_County            15837 non-null  int64  
 7   Taxpayer_Phone             15837 non-null  object 
 8   Location_Number            15837 non-null  int64  
 9   Location_Name              15837 non-null  object 
 10  Location_Address           15837 non-null  object 
 11  Location_City              15837 non-null  object 
 12  Location_State             15837 non-null  object 
 13  Location_Zip               15837 non-null  int

In [21]:
with open(r'fullState2025.pkl','wb') as ofile:
    pickle.dump(df, ofile)

In [25]:
with open(r'fullState2024.pkl','rb') as ifile:
    df = pickle.load(ifile)

In [27]:
!date

Wed Jul  9 12:43:26 PM CDT 2025


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16107 entries, 0 to 16106
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Taxpayer_Number            16107 non-null  int64  
 1   Taxpayer_Name              16107 non-null  object 
 2   Taxpayer_Address           16107 non-null  object 
 3   Taxpayer_City              16107 non-null  object 
 4   Taxpayer_State             16107 non-null  object 
 5   Taxpayer_Zip               16107 non-null  object 
 6   Taxpayer_County            16107 non-null  int64  
 7   Taxpayer_Phone             16107 non-null  object 
 8   Location_Number            16107 non-null  int64  
 9   Location_Name              16107 non-null  object 
 10  Location_Address           16107 non-null  object 
 11  Location_City              16107 non-null  object 
 12  Location_State             16107 non-null  object 
 13  Location_Zip               16107 non-null  int

In [ ]:
!pip3 install folium

In [29]:
!date

Wed Jul  9 12:43:59 PM CDT 2025


In [30]:
import folium

# Create the base map centered on San Antonio
hotel_map = folium.Map(location=[29.5187, -98.6047], zoom_start=12)

# Normalize receipts to set radius scale
min_receipt = df["Total_Room_Receipts"].min()
max_receipt = df["Total_Room_Receipts"].max()
range_receipt = max_receipt - min_receipt

# Add scaled circle markers
for _, row in df.dropna(subset=["Latitude", "Longitude"]).iterrows():
    # Scale radius between 4 and 20
    scaled_radius = 4 + 16 * ((row["Total_Room_Receipts"] - min_receipt) / range_receipt)

    # Color by receipt amount (simple logic: low = light, high = dark blue)
    if row["Total_Room_Receipts"] == 0:
        color = "gray"
    elif row["Total_Room_Receipts"] < 100000:
        color = "lightblue"
    elif row["Total_Room_Receipts"] < 300000:
        color = "blue"
    else:
        color = "darkblue"

    popup_text = f"""
    <b>{row['Location_Name']}</b><br>
    Receipts: ${row['Total_Room_Receipts']:,.2f}<br>
    Units: {row['Unit_Capacity']}
    """

    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=scaled_radius,
        color=color,
        fill=True,
        fill_color=color,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(hotel_map)

# Save the map
hotel_map.save("Texas_May24_receipts_scaled_map_2.html")


In [31]:
!date

Wed Jul  9 12:44:08 PM CDT 2025


In [ ]:
!ls
!pwd